In [ ]:
# Mann-Kendal Test for each month separately across years (e.g. all Januaries from 1980 to 2017).

from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pymannkendall as mk
import numpy as np


rain_data = pd.read_excel("rainfall.xlsx", sheet_name="Sheet1")  # Import Excel file

# Reshape from wide to long format to separate months across years
rain_data_long = rain_data.melt(id_vars="Years", var_name="Month", value_name="Rainfall")

# Ensure months are in correct order
month_order = ["January", "February", "March", "April", "May", "June",
               "July", "August", "September", "October", "November", "December"]

rain_data_long["Month"] = pd.Categorical(rain_data_long["Month"], categories=month_order, ordered=True)

# Apply Mann-Kendall test for each month separately
results = {}
for month in month_order:
    # Filter data for the specific month
    month_data = rain_data_long[rain_data_long["Month"] == month]
    
    # Perform Mann-Kendall test
    result = mk.original_test(month_data["Rainfall"])
    
    # Store the result
    results[month] = {
        "Trend": result.trend,
        "p-value": result.p,
        "Tau": result.Tau,
        "Slope": result.slope,
        "Intercept": result.intercept
    }
    
    # Plot each month's trend
    plt.figure(figsize=(12, 6))
    plt.plot(month_data["Years"], month_data["Rainfall"], marker='o', color='blue')
    plt.title(f"{month} Rainfall Trend (1980–2017)", fontsize=16)
    plt.xlabel("Year")
    plt.ylabel("Rainfall (mm)")
    plt.grid(True)
    plt.tight_layout()

# Fit and plot trendline (linear regression)
    x = month_data["Years"]
    y = month_data["Rainfall"]
    coeffs = np.polyfit(x, y, deg=1)
    trendline = np.poly1d(coeffs)
    plt.plot(x, trendline(x), color='red', linestyle='--', linewidth=2, label="Trendline")
    
   # Save the plot as a PNG file in the output folder
    output_folder = "output"
    os.makedirs(output_folder, exist_ok=True)  # Create an output folder if it doesn’t exist
    plt.savefig(os.path.join(output_folder, f"{month}_monthly_mann_kendall_test.png"))
    #plt.close()  # Close the figure to avoid display (optional)

# Display the results for each month
for month, result in results.items():
    print(f"{month}:")
    print(f"  Trend: {result['Trend']}")
    print(f"  p-value: {result['p-value']}")
    print(f"  Kendall's Tau: {result['Tau']}")
    print(f"  Slope: {result['Slope']}")
    print(f"  Intercept: {result['Intercept']}")
    print("-" * 50)
 